Before launching jupyter and this notebook, type in terminal: 

source activate nbodykit-env

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from nbodykit.lab import *
from nbodykit import setup_logging, style
from nbodykit.algorithms.fftpower import project_to_basis

import os
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(style.notebook)

In [3]:
setup_logging() # turn on logging to screen

# FKP norm

Let us compte the normalization of the FKP estimator. 

My strategy is to call exactly the same functions from *nbodykit* that are called when computing the power spectrum so that we are assured that we get the right answer: after all, we already computed the right normalization, $A_{s\rightarrow0}$, taking the $s\rightarrow 0$ limit of the window function that is simply the Fourier transform of the power spectrum of the randoms. 

Here are the definitions for the FKP normalization we know:
\begin{align}
A & \equiv \int d^3 r \ n_w(r)^2 \\
A_{s\rightarrow 0} & = \lim_{s\rightarrow 0} Q(s) = \lim_{s\rightarrow 0} \int d^3 r \ n_w(r) n_w(r-s)\\
A_{k \rightarrow 0} & = \lim_{k\rightarrow 0} \int d^3 r \ e^{-ikr} n_w(r)^2 \ ,
\end{align}
where $n_w \equiv n \cdot w$ is a shorthand. 

We already computed $A_{s\rightarrow 0}$. Let us try $A_{k \rightarrow 0}$. 

\begin{align}
A_{k \rightarrow 0} & = \lim_{k\rightarrow 0} \int d^3 r \ e^{-ikr} n_w(r)^2 \\
 & = \lim_{k\rightarrow 0} \int d^3 p \ \tilde n_w(p+k) \tilde n_w(p) \\
 & = \int d^3 p \ \tilde n_w(p)^2 \ ,
\end{align}
where we use the convolution theorem at second line, with $\tilde n_w(p) = \int d^3 r \ e^{-ipr} n_w(r)$. 

So in fact we can now see the relationship between the two definitions: 
\begin{align}
A_{s\rightarrow 0} & = \lim_{s\rightarrow 0} \int d^3 p \ e^{iks} \tilde n_w(p)^2 \\
 & = \lim_{s\rightarrow 0} \int d^3 r \ n_w(r-s) n_w(r) \\
 & = \int d^3 r \ n_w(r)^2 \ ,
\end{align}
using once again the convolution theorem at second line, with $n_w(r) = \int d^3 p \ e^{ipr} \tilde n_w(p)$. 


### Getting BOSS catalogs

In [4]:
def print_download_progress(count, block_size, total_size):
    import sys
    pct_complete = float(count * block_size) / total_size
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)
    sys.stdout.write(msg)
    sys.stdout.flush()

def download_data(download_dir):
    """
    Download the FITS data needed for this notebook to the specified directory.
    
    Parameters
    ----------
    download_dir : str
        the data will be downloaded to this directory
    """
    from six.moves import urllib
    import shutil
    import gzip
    
#     urls = ['https://data.sdss.org/sas/dr12/boss/lss/galaxy_DR12v5_LOWZ_South.fits.gz',
#             'https://data.sdss.org/sas/dr12/boss/lss/random0_DR12v5_LOWZ_South.fits.gz']
#     filenames = ['galaxy_DR12v5_LOWZ_South.fits', 'random0_DR12v5_LOWZ_South.fits']
#     urls = ['https://data.sdss.org/sas/dr12/boss/lss/galaxy_DR12v5_CMASS_North.fits.gz',
#             'https://data.sdss.org/sas/dr12/boss/lss/random0_DR12v5_CMASS_North.fits.gz']
#     filenames = ['galaxy_DR12v5_CMASS_North.fits', 'random0_DR12v5_CMASS_North.fits']
    urls = ['https://data.sdss.org/sas/dr12/boss/lss/random0_DR12v5_CMASSLOWZTOT_North.fits.gz',
            'https://data.sdss.org/sas/dr12/boss/lss/random0_DR12v5_CMASSLOWZTOT_South.fits.gz',
            'https://data.sdss.org/sas/dr12/boss/lss/galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz',
            'https://data.sdss.org/sas/dr12/boss/lss/galaxy_DR12v5_CMASSLOWZTOT_South.fits.gz']
    filenames = ['random0_DR12v5_CMASSLOWZTOT_North.fits', 
                 'random0_DR12v5_CMASSLOWZTOT_South.fits',
                 'galaxy_DR12v5_CMASSLOWZTOT_North.fits', 
                 'galaxy_DR12v5_CMASSLOWZTOT_South.fits']
    
    # download both files
    for i, url in enumerate(urls):
        
        # the download path
        filename = url.split('/')[-1]
        file_path = os.path.join(download_dir, filename)
        final_path = os.path.join(download_dir, filenames[i])
        
        # do not re-download
        if not os.path.exists(final_path):
            print("Downloading %s" % url)
            
            # Check if the download directory exists, otherwise create it.
            if not os.path.exists(download_dir):
                os.makedirs(download_dir)

            # Download the file from the internet.
            file_path, _ = urllib.request.urlretrieve(url=url,
                                                      filename=file_path,
                                                      reporthook=print_download_progress)

            print()
            print("Download finished. Extracting files.")

            # unzip the file
            with gzip.open(file_path, 'rb') as f_in, open(final_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
            os.remove(file_path)
            print("Done.")
        else:
            print("Data has already been downloaded.")

In [5]:
# download the data to the current directory
download_dir = "catalogs"
download_data(download_dir)

- Download progress: 1.8%

KeyboardInterrupt: 

In [ ]:
# NOTE: change this path if you downloaded the data somewhere else!
# data_path = os.path.join(download_dir, 'galaxy_DR12v5_LOWZ_South.fits')
# randoms_path = os.path.join(download_dir, 'random0_DR12v5_LOWZ_South.fits')
# data_path = os.path.join(download_dir, 'galaxy_DR12v5_CMASSLOWZTOT_North.fits')
# randoms_path = os.path.join(download_dir, 'random0_DR12v5_CMASSLOWZTOT_North.fits')
data_path = os.path.join(download_dir, 'galaxy_DR12v5_CMASSLOWZTOT_South.fits')
randoms_path = os.path.join(download_dir, 'random0_DR12v5_CMASSLOWZTOT_South.fits')

# initialize the FITS catalog objects for data and randoms
data = FITSCatalog(data_path)
randoms = FITSCatalog(randoms_path)

We can analyze the available columns in the catalogs via the ``columns`` attribute:

In [ ]:
print('data columns = ', data.columns)

In [ ]:
print('randoms columns = ', randoms.columns)

### Setting the catalogs exactly as in our $P(k)$ measurements

In [ ]:
ZMIN = 0.2
ZMAX = 0.43
# ZMIN = 0.43
# ZMAX = 0.7

# slice the randoms
valid = (randoms['Z'] > ZMIN)&(randoms['Z'] < ZMAX)
randoms = randoms[valid]

# slice the data
valid = (data['Z'] > ZMIN)&(data['Z'] < ZMAX)
data = data[valid]

In [ ]:
# the fiducial BOSS DR12 cosmology
cosmo = cosmology.Cosmology(h=0.676).match(Omega0_m=0.31)

# add Cartesian position column
data['Position'] = transform.SkyToCartesian(data['RA'], data['DEC'], data['Z'], cosmo=cosmo)
randoms['Position'] = transform.SkyToCartesian(randoms['RA'], randoms['DEC'], randoms['Z'], cosmo=cosmo)

In [ ]:
randoms['WEIGHT'] = 1.0
data['WEIGHT'] = data['WEIGHT_SYSTOT'] * (data['WEIGHT_NOZ'] + data['WEIGHT_CP'] - 1.0)

### Wrong normalization $A_\Sigma$ from standard nbodykit $P(k)$ measurements pipeline

Here we create a mesh and computes the power spectrum multipoles of BOSS.  
This operation will give us the `wrong' normalization, $A_\Sigma$, that we can compare to. 

In [ ]:
# combine the data and randoms into a single catalog
fkp = FKPCatalog(data, randoms)

In [ ]:
# As we care only getting the `wrong' normalization, 
# here we just put minimal options so that the two following operations are fast
mesh = fkp.to_mesh(Nmesh=16, BoxSize=3500., 
    nbar='NZ', fkp_weight='WEIGHT_FKP', comp_weight='WEIGHT', resampler='NEAREST', interlaced=False)

In [ ]:
r = ConvolvedFFTPower(mesh, poles=[0], kmin=0.)

In [ ]:
for key in r.attrs:
    print("%s = %s" % (key, str(r.attrs[key])))

data.norm / randoms.norm above are $A_\Sigma$. We call it `I22` in Python. 

In [ ]:
I22 = r.attrs['randoms.norm']
alpha = r.attrs['alpha']

### Wrong normalization $A_\Sigma$  from scratch

We compute $A_\Sigma$ ourself now: 

\begin{equation}
A_\Sigma = \sum_i^{N_p} \bar n_i w_i^2 \ ,
\end{equation}

We are going to use *nbodykit* internal functions to make sure they do what we think they do. We furthermore put the $N_p$ objects of the (random) catalog on a grid to make sure that the grid is not making anything weird. 

But first let us do it ourself at the level of the catalog to make sure. 

In [ ]:
norm_data = np.sum(data['NZ'].compute() * data['WEIGHT'].compute() * data['WEIGHT_FKP'].compute()**2)
norm_randoms = np.sum(randoms['NZ'].compute() * randoms['WEIGHT'].compute() * randoms['WEIGHT_FKP'].compute()**2)

print ('%.3f' % (norm_data / I22))
print ('%.3f' % (alpha * norm_randoms / I22))

Let's do it with *nbodykit*

In [ ]:
randoms['WEIGHT_FKP_squared'] = 1. * randoms['WEIGHT_FKP']**2
randoms['WEIGHT_squared'] = 1. * randoms['WEIGHT']**2 * randoms['NZ']

In [ ]:
fkp = FKPCatalog(randoms, None, BoxPad=0.) 
# fkp = FKPCatalog(data, None, BoxPad=0.) ### fast but with cosmic variance

In [ ]:
Nmesh, Lbox = 128, 3500. # this is whatever, as long as the box is large enough
mesh = fkp.to_mesh(Nmesh=Nmesh, BoxSize=Lbox, 
                   nbar='NZ', fkp_weight='WEIGHT_FKP_squared', comp_weight='WEIGHT_squared', 
                   resampler='NEAREST', interlaced=False) 

3D field of $n w^2$ in configuration space:

In [ ]:
nw2 = mesh.to_real_field() 

Sum over the whole box in configuration space:

In [ ]:
muedges = np.linspace(-1, 1, 2, endpoint=True)
xedges = np.array([0., Lbox]) 
edges = [xedges, muedges]

### result = (xmean_2d, mumean_2d, y2d, N_2d)
proj_result, _ = project_to_basis(nw2, edges)

We check that we get the same: 

In [ ]:
norm = alpha * np.real(np.squeeze(proj_result[2])) * Lbox**3
print ('%.3f' % (norm / I22))

the factor of $\alpha^1$ is to rescale our results to the correct one as we use $1$ random field.  
the factor of $V = L_{\rm box}^3$ is what *nbodykit* does, so I do it. 

Notice that we get the same is really a good check: *nbodykit* really does the sum over the objects in the catalogs, while I instead put the re-weighted objects on a grid first then summed the cells. 

The other possibility to perform the sum is simply to Fourier transform the configuration-space field $nw^2$ and take the $0$-mode of the Fourier field: 

\begin{equation}
A_\Sigma = \lim_{k\rightarrow 0} \sum_i \ e^{-ik r} n_i w_i^2
\end{equation}

In [ ]:
nw2_k = nw2.r2c() 

Checking the same from the Fourier $0$-mode: 

In [ ]:
norm = alpha * np.real(nw2_k[0,0,0]) * Lbox**3
print ('%.3f' % (norm / I22)) 

## Let us do $A_{k\rightarrow 0}$ now!

Here we need to take a box big enough to make sure that we have all the modes.  
We alse need to take thin enough cells to resolve the modes down to the scales where the `window' is constant (just acting like an identity function).  
Therefore, this operation might be memory-intensive. 

In [ ]:
Nmesh, Lbox = 256, 2000. # on my laptop I can not take too big Nmesh
mesh = fkp.to_mesh(Nmesh=Nmesh, BoxSize=Lbox, 
    nbar='NZ', fkp_weight='WEIGHT_FKP', comp_weight='WEIGHT', 
    compensated=True, resampler='tsc', interlaced=True) 
# now we want to throw in some good options to resolve as deep as possible

3D field of $w$ in configuration space:

In [ ]:
w = mesh.to_real_field() 

It is tempting to first multiply $w$ by itself in configuration space then Fourier transform and look the Fourier $0$-mode as our definition of $A_{k\rightarrow 0}$ suggests:

\begin{equation}\label{eq:real}
A_{k\rightarrow 0} = \lim_{k\rightarrow 0} \int d^3 r \ e^{-ikr} n_w(r)^2
\end{equation}

However, this operation is UV sensitive, and importantly, not in the same way as the way we measure the power spectrum!

Indeed, for the power spectrum, we instead take the product of the fields in Fourier space. 

Therefore, we should instead use the convolution formulation: 

\begin{equation}\label{eq:fourier}
A_{k\rightarrow 0} = \lim_{k\rightarrow 0} \int d^3 p \ \tilde n_w(p-k) \tilde n_w(p) = \int d^3 p \ \tilde n_w(p)^2 \ ,
\end{equation}

where the products of the field are taken in Fourier space consistently with the way we measure the power spectrum. 

Still, let us first do the calculation where we square the field in configuration space to see what we get and then compare to the calculation where we square the field in Fourier space. 

##### $A$ from squaring the configuration-space field

We shall not forget to compensate for the interpolation kernels. 

So Let's Fourier transform first the $w$ field, compensate, then Fourier transform back. 

In [ ]:
w_k = w.r2c()

In [ ]:
def get_compensation(mesh):
    toret = None
    try:
        compensation = mesh._get_compensation()
        toret = {'func':compensation[0][1], 'kind':compensation[0][2]}
    except ValueError:
        pass
    return toret

In [ ]:
compensation = get_compensation(mesh)
print (compensation)

In [ ]:
wc_k = w_k.apply(out=Ellipsis, **compensation)

Fourier transforming back: 

In [ ]:
wc = wc_k.c2r()

Squaring in configuration space: 

In [ ]:
wc2_direct = 1.*wc
for islab in range(wc.shape[0]):
    wc2_direct[islab,...] = wc[islab]*wc[islab]

Now doing the integral over the whole configuration space by taking the $0$-mode in the reciprocal space: 

In [ ]:
wc2_direct_k = wc2_direct.r2c()

In [ ]:
norm = alpha**2 * np.real(wc2_direct_k[0,0,0]) * Lbox**3 # now it is alpha^2 because we have two randoms fields
print ('norm=%.3f, ratio=%.3f' % (norm, norm / I22)) 

##### $A$ from squaring the Fourier-space field

We can instead take the product of $w$ in Fourier space: 

In [ ]:
wc2_k = 1.*wc_k
for islab in range(wc_k.shape[0]):
    wc2_k[islab,...] = wc_k[islab]*wc_k[islab].conj()

And finally to perform the integral over the whole Fourier space we may use the trick where we take the $0$-mode of the reciprocal space: 

\begin{align}
A_{s\rightarrow 0} & = \lim_{s\rightarrow 0} \int d^3 p \ e^{iks} \tilde n_w(p)^2 \\
 & = \lim_{s\rightarrow 0} \int d^3 r \ n_w(r-s) n_w(r) \\
 & = \int d^3 r \ n_w(r)^2 \ ,
\end{align}

Notice that this exactly the way that we have already computed $A_{s\rightarrow 0}$ through the window functions!

Let us then Fourier transform back $\tilde w^2$ to configuration space: 

In [ ]:
wc2 = wc2_k.c2r()

Norm from configuration-space $0$-mode:

In [ ]:
norm = alpha**2 * np.real(wc2[0,0,0]) * Lbox**3 # now it is alpha^2 because we have two randoms fields
print ('norm=%.3f, ratio=%.3f' % (norm, norm / I22)) 

In [ ]:
print ('ratio real squared vs. Fourier squared = %.3f' % (np.real(wc2_direct_k[0,0,0]/wc2[0,0,0]))) 

Looks good so far, though, it is not the right answer... Why?

The correct normalization can not be 

The `right' answer (the one I get from the window functions) is $A_{\rm right}/A_{\rm wrong} \equiv A_{s\rightarrow 0} / A_\Sigma \sim 0.88 \pm 0.01$. 

- Nmesh, Lbox = 256, 2000.: norm=2.145, ratio=1.062 <-- too big  
- Nmesh, Lbox = 256, 3000.: norm=1.932, ratio=0.956 <-- still too big, we are still not taking enough large box??? 
(Indeed, when getting the normalization through the window function, I actually take a box of Lbox = 100000...)  
- Nmesh, Lbox = 256, 10000.: norm=1.783, ratio=0.883 <-- for LOWZ SGC, could it be that we are taking large enough box now??? 
- Nmesh, Lbox = 256, 10000.: norm=1.844, ratio=0.913 <-- Mmmh, looks like this is not about being not large enough, rather we were cutting modes from the deep regime where the window is constant...

So, as expected, the results are quite dependent on if we are able to include all modes in the box (large `Lbox`) and resolve enough the UV (large `Nmesh`). We should at least the same range that I do when I am getting the normalization through the window function. 

### Last comments...

1. Maybe we should take care first of the shot noise (of the randoms) that will shift the value of 
$A_{s\rightarrow 0} = \lim_{s\rightarrow 0} \int d^3 p \ e^{iks} \tilde n_w(p)^2$ up. 

2. My fear is that we are sensitive to the junk $k>k_{\rm Nyq}$ in this procedure. This could explain why the get a too big normalization! (indeed in the computation through the window function I make sure to remove the junk in $Q(k)$ before integrating over to get the normalization).  

Therefore, we should not use the $0$-mode trick to do the integral over the Fourier space but stay in Fourier and do the integral in the region that we trust, that is, $k \in [k_f, \sim k_{\rm Nyq}]$, with $k_f = 2pi/L_{\rm box}, k_{\rm Nyq} = \pi N_{\rm mesh}/L_{\rm box}$. 

Let us do that: 

In [ ]:
muedges = np.linspace(-1, 1, 2, endpoint=True)
kf, knyq = 2*np.pi / Lbox, np.pi * Nmesh / Lbox
print (kf, knyq)
# xedges = np.array([kf, knyq])
xedges = np.array([0, knyq*1000.]) ### for now let me just try to reproduce the $0$-mode results
edges = [xedges, muedges]

### result = (xmean_2d, mumean_2d, y2d, N_2d)
proj_result, _ = project_to_basis(wc2_k, edges)

In [ ]:
norm = alpha**2 * np.real(np.squeeze(proj_result[2])) * Lbox**3
print ('%.3f' % (norm / I22))

In [ ]:
norm_0 = alpha**2 * np.real(wc2[0,0,0]) * Lbox**3 # 0-mode results
norm_int = alpha**2 * np.real(np.squeeze(proj_result[2])) * (Lbox**3)**2 / (2*np.pi)**3 
# there are some factors here that I don't know, just putting random stuff now to try to get something reasonable
print (norm_int/norm_0) # OK this should work with the right factors!!!

Once this is clear let us go the super-computer to crank up `Nmesh`. 